# Prepare script for relax_dynamic:at_temp calculations

This Notebook is designed to call master prepare for relax_dynamic calculations for the at_temp branch. The Notebook provides a routine that allows for more dynamic control over which calculations are created:

- New calculations at higher temperatures are only prepared if the calculation at the next lowest temperature finished and the relaxed box dimensions appear to still be consistent with the original crystal family.
- Calculations are only set up for select prototype/reference families at a time to allow for different size mults based on different numbers of atoms in the unit cells.

In [1]:
import iprPy
import atomman as am
import numpy as np

## MISSING ALTERNATE LAMMPS_COMMANDS!!!!

In [2]:
database_name = 'master'
run_directory_name = 'master_6'

lammps_command = 'lmp_mpi'
#mpi_command = ''
np_per_runner = 1



In [3]:
database = iprPy.load_database(database_name)

In [20]:
pots_df = database.get_records_df('potential_LAMMPS', pair_style='eam')
pots_df

,name,id,key,url,potid,potkey,poturl,units,atom_style,allsymbols,pair_style,status,symbols,elements,artifacts,comments,dois
0,1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1,1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1,062d2ba7-3903-40ae-a772-daa471d107c6,062d2ba7-3903-40ae-a772-daa471d107c6,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,None,metal,atomic,False,eam,active,"[Cu, Ni]","[Cu, Ni]","[{'filename': 'Cu_smf7.eam', 'label': None, 'u...",Potential 1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr...,[10.1103/physrevb.32.7685]
1,1986--Foiles-S-M--Ag--LAMMPS--ipr1,1986--Foiles-S-M--Ag--LAMMPS--ipr1,76a265fc-45ff-49d7-8c64-2044f12402f2,76a265fc-45ff-49d7-8c64-2044f12402f2,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Ag,672d54f8-9f48-4200-af56-8a7378ebbc4a,None,metal,atomic,False,eam,active,[Ag],[Ag],"[{'filename': 'Ag_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Ag--LAMMPS--ipr1 l...,[10.1103/physrevb.33.7983]
2,1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1,1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1,c5afa7e8-6b3b-49cd-ad1c-ae3e4329363a,c5afa7e8-6b3b-49cd-ad1c-ae3e4329363a,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Ag-Au-Cu-...,7a1302de-59cf-4efb-900e-cad845b68ee5,None,metal,atomic,False,eam,active,"[Ag, Au, Cu, Ni, Pd, Pt]","[Ag, Au, Cu, Ni, Pd, Pt]","[{'filename': 'Ag_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt-...,[10.1103/physrevb.33.7983]
3,1986--Foiles-S-M--Au--LAMMPS--ipr1,1986--Foiles-S-M--Au--LAMMPS--ipr1,c588810a-b96d-4871-bfe2-cff8a5a7c709,c588810a-b96d-4871-bfe2-cff8a5a7c709,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Au,ffb66faa-319d-4556-8363-dad3959cd553,None,metal,atomic,False,eam,active,[Au],[Au],"[{'filename': 'Au_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Au--LAMMPS--ipr1 l...,[10.1103/physrevb.33.7983]
4,1986--Foiles-S-M--Cu--LAMMPS--ipr1,1986--Foiles-S-M--Cu--LAMMPS--ipr1,380d3b47-51e9-4590-8a59-8313dd8fb018,380d3b47-51e9-4590-8a59-8313dd8fb018,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Cu,7991d330-58cd-43ac-bba9-ff6a58dcf617,None,metal,atomic,False,eam,active,[Cu],[Cu],"[{'filename': 'Cu_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Cu--LAMMPS--ipr1 l...,[10.1103/physrevb.33.7983]
5,1986--Foiles-S-M--Ni--LAMMPS--ipr1,1986--Foiles-S-M--Ni--LAMMPS--ipr1,8e9ae12d-5034-418a-a168-fb5499ecffcd,8e9ae12d-5034-418a-a168-fb5499ecffcd,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Ni,15a085b9-d9d6-404f-9c2f-3ed40e8ff7a4,None,metal,atomic,False,eam,active,[Ni],[Ni],"[{'filename': 'Ni_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Ni--LAMMPS--ipr1 l...,[10.1103/physrevb.33.7983]
6,1986--Foiles-S-M--Pd--LAMMPS--ipr1,1986--Foiles-S-M--Pd--LAMMPS--ipr1,fa189ff3-4a27-4c36-b8b7-3821443a4edd,fa189ff3-4a27-4c36-b8b7-3821443a4edd,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Pd,16dde7ea-c8cf-4a23-95b3-494a2b252e9b,None,metal,atomic,False,eam,active,[Pd],[Pd],"[{'filename': 'Pd_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Pd--LAMMPS--ipr1 l...,[10.1103/physrevb.33.7983]
7,1986--Foiles-S-M--Pt--LAMMPS--ipr1,1986--Foiles-S-M--Pt--LAMMPS--ipr1,0a74d2a8-ec49-459c-903a-44bd3e50969a,0a74d2a8-ec49-459c-903a-44bd3e50969a,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Pt,87657840-e5e6-4378-94ea-381a90608142,None,metal,atomic,False,eam,active,[Pt],[Pt],"[{'filename': 'Pt_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Pt--LAMMPS--ipr1 l...,[10.1103/physrevb.33.7983]
8,1989--Adams-J-B--Ag--LAMMPS--ipr1,1989--Adams-J-B--Ag--LAMMPS--ipr1,cbe0bfac-4e4c-49e4-91a4-e6d360ff0516,cbe0bfac-4e4c-49e4-91a4-e6d360ff0516,1989--Adams-J-B-Foiles-S-M-Wolfer-W-G--Ag,d1e36148-f138-4ee0-ab7d-1fb5442805d8,None,metal,atomic,False,eam,active,[Ag],[Ag],"[{'filename': 'agu6.txt', 'label': None, 'url'...",Potential 1989--Adams-J-B--Ag--LAMMPS--ipr1 li...,[10.1557/jmr.1989.0102]
9,1989--Adams-J-B--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1,1989--Adams-J-B--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1,8a99e4c9-7a90-4c0e-96f5-57048066940f,8a99e4c9-7a90-4c0e-96f5-57048066940f,1989--Adams-J-B-Foiles-S-M-Wolfer-W-G--Ag-Au-C

In [27]:
parent_df = database.get_records_df('relaxed_crystal', method='dynamic', standing='good')
parent_df

,name,key,method,standing,family,parent_key,potential_LAMMPS_id,potential_LAMMPS_key,potential_id,potential_key,...,composition,a,b,c,alpha,beta,gamma,natoms,potential_energy,cohesive_energy
0,00016026-13c1-48a3-8e66-6442552cd816,00016026-13c1-48a3-8e66-6442552cd816,dynamic,good,L2_1--AlCu2Mn--heusler,b3634615-d6c4-40a9-81a9-96f68a4b5354,1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1,c5afa7e8-6b3b-49cd-ad1c-ae3e4329363a,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Ag-Au-Cu-...,7a1302de-59cf-4efb-900e-cad845b68ee5,...,Ni2PdPt,5.941781,5.941781,5.941781,90.0,90.000000,90.0,16,-4.752035,-4.752035
1,000250f6-3d0c-40fb-bd1d-e8875c28600d,000250f6-3d0c-40fb-bd1d-e8875c28600d,dynamic,good,B3--ZnS--cubic-zinc-blende,b30885b3-399e-4b10-9093-c37e5131f2fe,EAM_Dynamo_SchopfBrommerFrigan_2012_AlMnPd__MO...,MO_137572817842_000,2012--Schopf-D-Brommer-P-Frigan-B-Trebin-H-R--...,bdd76724-0693-4e46-8a9a-39e1bc2204ce,...,MnPd,6.123487,6.123487,6.123487,90.0,90.000000,90.0,8,-3.417662,-3.415680
2,00034db2-afba-42ac-980c-310090e19013,00034db2-afba-42ac-980c-310090e19013,dynamic,good,oqmd-1214849,a2329076-bcd7-4cda-a858-0786ebd73ea6,2018--Starikov-S-V--U-Mo--LAMMPS--ipr1,dde865b2-0583-4369-92fb-4319eabdc6f6,2018--Starikov-S-V-Kolotova-L-N-Kuksin-A-Y-et-...,0363c268-9d42-4d82-9cc7-5a610c707c6c,...,U,10.762897,10.762897,10.762897,90.0,90.000000,90.0,58,-4.151113,-4.151113
3,00047c7e-a368-49c3-a879-e3154e803dca,00047c7e-a368-49c3-a879-e3154e803dca,dynamic,good,C1--CaF2--fluorite,888db48a-93e1-41ab-8dd6-ae5777c5c5a7,2011--Bonny-G--Fe-Cr--LAMMPS--ipr3,e442c47b-fa74-4b6f-bba9-2549bfa5eea0,2011--Bonny-G-Pasianot-R-C-Terentyev-D-Malerba...,c42c513a-f843-41c4-ba4f-3f31316c6a7b,...,Cr2Fe,5.648375,5.648375,5.648375,90.0,90.000000,90.0,12,-3.270608,-3.270317
4,0008b170-7de7-4d95-8663-90d8c69acf58,0008b170-7de7-4d95-8663-90d8c69acf58,dynamic,good,mp-555583,92e876ff-8c06-481d-8cf0-7482c6e633b3,SW_ZhouWardMartin_2013_CdTeZnSeHgS__MO_5032611...,MO_503261197030_002,2013--Zhou-X-W-Ward-D-K-Martin-J-E-et-al--Zn-C...,95fe0b68-f5a1-4bb6-9a11-f446b361d12b,...,SZn,3.822623,3.822623,74.907504,90.0,90.000000,120.0,48,-3.080004,-3.080004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23849,fff49fd4-1089-4cb9-9828-e0e66bcd177b,fff49fd4-1089-4cb9-9828-e0e66bcd177b,dynamic,good,mp-1056079,80c00fb3-391f-41ab-83f2-b05b9780b00b,2003--Lee-B-J--Cu--LAMMPS--ipr1,d6eca59d-2431-43fc-bb18-40a00cb42014,2003--Lee-B-J-Shim-J-H-Baskes-M-I--Cu,e2f7ee59-fa26-496a-9463-550f8f7a9011,...,Cu,11.842895,2.309780,22.747576,90.0,101.935919,90.0,1,-1.966010,-1.966010
23850,fff50516-52b6-43ad-b1af-f9dde12be534,fff50516-52b6-43ad-b1af-f9dde12be534,dynamic,good,mp-1215527,0b44ef80-bcb8-4399-8812-a958df126f24,SW_ZhouWardMartin_2013_CdTeZnSeHgS__MO_5032611...,MO_503261197030_002,2013--Zhou-X-W-Ward-D-K-Martin-J-E-et-al--Zn-C...,95fe0b68-f5a1-4bb6-9a11-f446b361d12b,...,CdSeTeZn,4.265122,4.265122,21.202966,90.0,90.000000,120.0,12,-2.355650,-2.355650
23851,fff65987-ef88-4725-8047-ebbda1fa58ab,fff65987-ef88-4725-8047-ebbda1fa58ab,dynamic,good,oqmd-756634,8c24c0f5-8f36-4262-8eae-031470345cec,2009--Mendelev-M-I--Cu-Zr--LAMMPS--ipr1,099b21ab-7a0f-417a-a475-ea73bb213a11,2009--Mendelev-M-I-Kramer-M-J-Ott-R-T-et-al--C...,48d79068-e994-4858-b7d5-031b1f570dc3,...,CuZr,3.448812,4.457442,8.824259,90.0,90.000000,90.0,8,-4.866263,-4.866263
23852,fffb7663-46e0-4bd1-b745-f6a5e22b8f17,fffb7663-46e0-4bd1-b745-f6a5e22b8f17,dynamic,good,oqmd-1216031,522d26dc-bc1d-44f7-ad73-9208bda627c0,2021--Deluigi-O-R--Fe-Ni-Cr-Co-Cu--LAMMPS--ipr1,00976d5c-140b-417f-b0db-694ddc5cf8f2,2021--Deluigi-O-R-Pasianot-R-C-Valencia-F-J-et...,f9d8adb8-1bcf-442d-9f1c-130a94f32b16,...,Co,2.504309,2.504309,18.720671,90.0,90.000000,120.0,9,-4.408258,-4.408258


In [17]:
for natoms in np.unique(df.natoms):
    pass
print(natoms)

576


In [24]:
sizemult = {}
for natoms in range(1, 577):
    for i in range(17):
        if natoms * i * i * i >= 4000:
            break
    if not i in sizemult:
        sizemult[i] = [natoms]
    else:
        sizemult[i].append(natoms)

In [25]:
sizemult

{16: [1],
 13: [2],
 12: [3],
 10: [4, 5],
 9: [6, 7],
 8: [8, 9, 10, 11],
 7: [12, 13, 14, 15, 16, 17, 18],
 6: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
 5: [32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62],
 4: [63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148],
 3: [149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
 

In [ ]:
sdict = {}
sdict[16] = [1, 1]
sdict[13] = [2, 2]
sdict[12] = [3, 3]
sdict[10] = [4, 5]
sdict[9] = [6, 7]
sdict[8] = [8, 11]
sdict[7] = [12, 18]
sdict[6] = [19, 31]
sdict[5] = [32, 62]
sdict[4] = [63, 148]
sdict[3] = [149, 499]
sdict[2] = [500, 1000]

In [18]:
natoms = 576
for i in range(17):
    if natoms * i * i * i >= 4000:
        break
print(f'{i} {i} {i}')
print(natoms * i * i * i)

2 2 2
4608


In [28]:
sdict = {}
sdict[16] = [1, 1]
sdict[13] = [2, 2]
sdict[12] = [3, 3]
sdict[10] = [4, 5]
sdict[9] = [6, 7]
sdict[8] = [8, 11]
sdict[7] = [12, 18]
sdict[6] = [19, 31]
sdict[5] = [32, 62]
sdict[4] = [63, 148]
sdict[3] = [149, 499]
sdict[2] = [500, 1000]


for s in sdict:
    nmin, nmax = sdict[s]
    
    parent_natoms_df = parent_df[(parent_df.natoms >= nmin) & (parent_df.natoms <= nmax)]
    print(len(parent_natoms_df))

622
3400
137
4624
540
5294
3226
1865
1488
2258
394
6


### Prepare 50K relaxations

First, relaxation calculations at 50K are prepared for all relaxed_crystal records from the dynamic method with good standing.

In [3]:
# Load relaxed_crystal records
relaxed_df = database.get_records_df('relaxed_crystal', method='dynamic', standing='good',
                                     family=family,
                                     symbols=symbols)

# Skip OpenKIM potentials (for local Windows runs)
def get_mo(series):
    return series.potential_LAMMPS_key[:3] == 'MO_'
relaxed_df = relaxed_df[~relaxed_df.apply(get_mo, axis=1)]

parent_keys = relaxed_df.key.tolist()

database.master_prepare(
    styles = 'relax_dynamic:at_temp',
    np_per_runner = np_per_runner,
    run_directory = run_directory_name,
    num_pots = 900,
    lammps_command = 'lmp_mpi',
    temperature = '50',
    parent_key = parent_keys,
    sizemults = sizemults)

100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:08<00:00, 53.65it/s]


793 potential ids found
Preparing calculation relax_dynamic branch at_temp
Using potential #s 0 to 599

109137 existing calculation records found
600 matching interatomic potentials found
73 matching atomic parents found
73 calculation combinations to check
0 new records to prepare


preparing: 0it [00:00, ?it/s]


Using potential #s 600 to 792



109137 existing calculation records found
429 matching interatomic potentials found
0 matching atomic parents found
1 invalid calculations skipped
0 calculation combinations to check



### Prepare > 50K relaxations

1. All finished relax_dynamic calculations are retrieved.
2. Loop over all temperatures every 50K from 100K to 3000K
3. Identify all finished calculations for 50K less than the target temperature.
4. Check if the dimensions of the relaxed unit cell is within rtol of the expected crystal family.
5. Prepare new relaxation calculations at the target temperature from the associated relaxed_crystal record.

In [ ]:
# Method parameters
rtol = 0.05
crystal_family = 'cubic'
temperatures = np.arange(100, 3050, 50, dtype=int)

In [ ]:
def check_crystal_family(df, crystal_family, atol=0.0, rtol=0.05):
    """
    Checks if the 'box' field of df, which should be an atomman.Box object,
    is of the indicated crystal family within rtol.
    """
    
    # Define box_parameters apply function based on crystal_family
    
    if crystal_family == 'cubic':
        def box_parameters(series):
            """Check values and build the box_parameter term for cubic systems"""
            if series.box.iscubic(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b, series.box.c])
                return f'{a} {a} {a}'
            else:
                return np.nan
    
    elif crystal_family == 'hexagonal':
        def box_parameters(series):
            """Check values and build the box_parameter term for hexagonal systems"""
            if series.box.ishexagonal(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b])
                c = series.box.c
                return f'{a} {a} {c} 90.0 90.0 120.0'
            else:
                return np.nan
            
    elif crystal_family == 'tetragonal':     
        def box_parameters(series):
            """Check values and build the box_parameter term for tetragonal systems"""
            if series.box.istetragonal(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b])
                c = series.box.c
                return f'{a} {a} {c}'
            else:
                return np.nan
    
    elif crystal_family == 'rhombohedral':
        def box_parameters(series):
            """Check values and build the box_parameter term for rhombohedral systems"""
            if series.box.isrhombohedral(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b, series.box.c])
                alpha = np.mean([series.box.alpha, series.box.beta, series.box.gamma])
                return f'{a} {a} {a} {alpha} {alpha} {alpha}'
            else:
                return np.nan
        
    elif crystal_family == 'orthorhombic':
        def box_parameters(series):
            """Check values and build the box_parameter term for orthorhombic systems"""
            if series.box.isorthorhombic(atol=atol, rtol=rtol):
                a = series.box.a
                b = series.box.b
                c = series.box.c
                return f'{a} {b} {c}'
            else:
                return np.nan
        
    elif crystal_family == 'monoclinic':
        def box_parameters(series):
            """Check values and build the box_parameter term for monoclinic systems"""
            if series.box.ismonoclinic(atol=atol, rtol=rtol):
                a = series.box.a
                b = series.box.b
                c = series.box.c
                beta = series.box.beta
                return f'{a} {b} {c} 90.0 {beta} 90.0'
            else:
                return np.nan
            
    elif crystal_family == 'triclinic':
        def box_parameters(series):
            """Check values and build the box_parameter term for triclinic systems"""
            if series.box.istriclinic(atol=atol, rtol=rtol):
                a = series.box.a
                b = series.box.b
                c = series.box.c
                alpha = series.box.alpha
                beta = series.box.beta
                gamma = series.box.gamma
                return f'{a} {b} {c} {alpha} {beta} {gamma}'
            else:
                return np.nan
            
    else:
        raise ValueError(f'unknown crystal family: {crystal_family}')
    
    # Use the apply function on the dataframe
    df['box_parameters'] = df.apply(box_parameters, axis=1)
        
    # Return only the dataframe entries with good box_parameters
    return df[df.box_parameters.notna()]

In [8]:
# Fetch all currently finished relax_dynamic:at_temp calculations
relaxed_df = database.get_records_df('calculation_relax_dynamic', branch='at_temp', status='finished',
                                     family=family)

# Create a box object from average lengths and tilts
def make_box(series):
    """Create cell Box from relaxed lengths and tilts"""
    return am.Box(lx=series.lx, ly=series.ly, lz=series.lz,
                  xy=series.xy, xz=series.xz, yz=series.yz)
relaxed_df['box'] = relaxed_df.apply(make_box, axis=1)

for temperature in temperatures:
    print(temperature)
    
    # Get finished calculations for the previous temperature
    previous_temperature = temperature - 50
    previous_df = relaxed_df[relaxed_df.temperature == previous_temperature].reset_index(drop=True)
    if len(previous_df) == 0:
        print('No good parents found')
        continue

    # Check if relaxed box is still the correct crystal family, within rtol.
    previous_df = check_crystal_family(previous_df, crystal_family, rtol=rtol)

    # Get good relaxed_crystal parent keys
    parent_keys = previous_df.parent_key.tolist()
    if len(parent_keys) == 0:
        print('No good parents found')
        continue
    print(len(parent_keys), 'good parents found')
    
    # Prepare calculations at the temperature for the good parents
    database.master_prepare(
        styles = 'relax_dynamic:at_temp',
        np_per_runner = np_per_runner,
        run_directory = run_directory_name,
        num_pots = 900,
        lammps_command = lammps_command,
        temperature = str(temperature),
        parent_key = parent_keys,
        sizemults = sizemults)

1050
6 good parents found


100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:08<00:00, 51.40it/s]


793 potential ids found
Preparing calculation relax_dynamic branch at_temp
Using potential #s 0 to 792

109222 existing calculation records found
1029 matching interatomic potentials found
6 matching atomic parents found
6 calculation combinations to check
6 new records to prepare


preparing: 100%|#########################################################################| 6/6 [00:17<00:00,  2.99s/it]



1100
No good parents found
1150
6 good parents found


...


2100
No good parents found
2150
1 good parents found


100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:07<00:00, 55.41it/s]


793 potential ids found
Preparing calculation relax_dynamic branch at_temp
Using potential #s 0 to 792

109256 existing calculation records found
1029 matching interatomic potentials found
1 matching atomic parents found
1 calculation combinations to check
1 new records to prepare


preparing: 100%|#########################################################################| 1/1 [00:03<00:00,  3.09s/it]


2200
No good parents found
2250
No good parents found
2300
No good parents found
2350
No good parents found
2400
No good parents found
2450
No good parents found
2500
No good parents found
2550
No good parents found
2600
No good parents found
2650
No good parents found
2700
No good parents found
2750
No good parents found
2800
No good parents found
2850
No good parents found
2900
No good parents found
2950
No good parents found
3000
No good parents found
